In [1]:
from sympy import Function, Symbol, E, Integral, diff, limit, sqrt, simplify, series, pi, oo, erf, cosh, ln
from sympy.stats import Normal, cdf

In [2]:
delta = Symbol('Delta', positive=True)
l = Symbol('l', positive=True)
tau = Symbol('tau', positive=True)
theta = Symbol('theta', positive=True)
mu = Symbol('mu')
sig = Symbol('sigma', positive=True)
f = Symbol('f', positive=True)

In [3]:
# tick width
delta

Delta

In [4]:
# liquidity deployed per unit of external virtual reserves
l

l

In [5]:
# rebalancing period
tau

tau

In [6]:
# pool fee volume per unit of external virtual reserves
theta

theta

In [7]:
# pool fee rate
f

f

In [8]:
# log-normal drift
mu

mu

In [9]:
# log-normal vol
sig

sigma

In [10]:
# log-normal drift adjusted for vol
mu_p = mu - sig**2 / 2
mu_p

mu - sigma**2/2

In [11]:
# standard normal RV
Z = Normal('Z', 0, 1)
Z

Z

In [12]:
# alias standard norm cdf: Phi(z)
Phi = cdf(Z)

In [13]:
# "normalized" delta (+)
d_plus = (delta - mu_p * tau) / (sig * sqrt(tau))
d_plus

(Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau))

In [14]:
# "normalized" delta (-)
d_minus = (-delta - mu_p * tau) / (sig * sqrt(tau))
d_minus

(-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau))

In [15]:
# term for fee revenue EV (integral unevaluated)
t = Symbol('t')
d_plus_t = (delta - mu_p * t) / (sig * sqrt(t))
d_minus_t = (-delta - mu_p * t) / (sig * sqrt(t))

integrand_fee = Phi(d_plus_t) - Phi(d_minus_t) + E**(mu * tau) * ( Phi(d_plus_t - sig * sqrt(t)) - Phi(d_minus_t - sig * sqrt(t)) )

ev_fee = (theta / (1-E**(-delta/2) + l)) * Integral(integrand_fee, (t, 0, tau))
ev_fee

theta*Integral((-erf(sqrt(2)*(-sigma*sqrt(t) + (-Delta - t*(mu - sigma**2/2))/(sigma*sqrt(t)))/2)/2 + erf(sqrt(2)*(-sigma*sqrt(t) + (Delta - t*(mu - sigma**2/2))/(sigma*sqrt(t)))/2)/2)*exp(mu*tau) - erf(sqrt(2)*(-Delta - t*(mu - sigma**2/2))/(2*sigma*sqrt(t)))/2 + erf(sqrt(2)*(Delta - t*(mu - sigma**2/2))/(2*sigma*sqrt(t)))/2, (t, 0, tau))/(l + 1 - exp(-Delta/2))

In [16]:
# check evaluated integral from wolfram
x = Symbol('x')  # placeholders
a = Symbol('a', positive=True)
b = Symbol('b')

In [17]:
# int dx erf((a-bx)/sqrt(2x))
def ev_fee_integral_fn_1a(x, a, b):
    return (1/(2*b**2)) * (E**(2*a*b)*erf((a+b*x) / sqrt(2*x)) - E**(2*a*b) + 1 + (2*a*b + 1) * erf((b*x-a)/sqrt(2*x)))

In [18]:
def ev_fee_integral_fn_1b(x, a, b):
    return x * erf((a-b*x) / sqrt(2*x)) - sqrt(2*x/pi) * (1/b) * E**(-(a-b*x)**2/(2*x))

In [19]:
def ev_fee_integral_fn_1(x, a, b):
    return ev_fee_integral_fn_1a(x, a, b) + ev_fee_integral_fn_1b(x, a, b)

In [20]:
ev_fee_integral_fn_1(x, a, b)

x*erf(sqrt(2)*(a - b*x)/(2*sqrt(x))) - sqrt(2)*sqrt(x)*exp(-(a - b*x)**2/(2*x))/(sqrt(pi)*b) + ((2*a*b + 1)*erf(sqrt(2)*(-a + b*x)/(2*sqrt(x))) + exp(2*a*b)*erf(sqrt(2)*(a + b*x)/(2*sqrt(x))) - exp(2*a*b) + 1)/(2*b**2)

In [21]:
# checks out for - integral
simplify(diff(ev_fee_integral_fn_1(x, a, b), x))

erf(sqrt(2)*(a - b*x)/(2*sqrt(x)))

In [22]:
limit(ev_fee_integral_fn_1(x, a, b), x, 0)

-a/b

In [23]:
# int dx erf((a+bx)/sqrt(2x))
def ev_fee_integral_fn_2a(x, a, b):
    return (1/(2*b**2)) * E**(-2*a*b) * ( -erf((b*x-a)/sqrt(2*x)) + (2*a*b - 1) * (E**(2*a*b) * erf((a+b*x)/sqrt(2*x)) - E**(2*a*b) + 1) )

In [24]:
def ev_fee_integral_fn_2b(x, a, b):
    return x * erf((a+b*x)/sqrt(2*x)) + sqrt(2*x/pi) * (1/b) * E**(-(a+b*x)**2/(2*x))

In [25]:
def ev_fee_integral_fn_2(x, a, b):
    return ev_fee_integral_fn_2a(x, a, b) + ev_fee_integral_fn_2b(x, a, b)

In [26]:
ev_fee_integral_fn_2(x, a, b)

x*erf(sqrt(2)*(a + b*x)/(2*sqrt(x))) + sqrt(2)*sqrt(x)*exp(-(a + b*x)**2/(2*x))/(sqrt(pi)*b) + ((2*a*b - 1)*(exp(2*a*b)*erf(sqrt(2)*(a + b*x)/(2*sqrt(x))) - exp(2*a*b) + 1) - erf(sqrt(2)*(-a + b*x)/(2*sqrt(x))))*exp(-2*a*b)/(2*b**2)

In [27]:
# checks out for + integral
simplify(diff(ev_fee_integral_fn_2(x, a, b), x))

erf(sqrt(2)*(a + b*x)/(2*sqrt(x)))

In [28]:
limit(ev_fee_integral_fn_2(x, a, b), x, 0)

a*exp(-2*a*b)/b

In [29]:
def ev_fee_integral_fn(x, a, b):
    return ev_fee_integral_fn_1(x, a, b) / 2 + ev_fee_integral_fn_2(x, a, b) / 2

In [30]:
simplify(diff(ev_fee_integral_fn(x, a, b), x))

erf(sqrt(2)*(a - b*x)/(2*sqrt(x)))/2 + erf(sqrt(2)*(a + b*x)/(2*sqrt(x)))/2

In [31]:
ev_fee_a = (theta / (1-E**(-delta/2) + l)) * (ev_fee_integral_fn(tau, delta/sig, mu_p/sig) - limit(ev_fee_integral_fn(t, delta/sig, mu_p/sig), t, 0))
ev_fee_a

theta*(sigma**2*((2*Delta*(mu - sigma**2/2)/sigma**2 - 1)*(exp(2*Delta*(mu - sigma**2/2)/sigma**2)*erf(sqrt(2)*(Delta/sigma + tau*(mu - sigma**2/2)/sigma)/(2*sqrt(tau))) - exp(2*Delta*(mu - sigma**2/2)/sigma**2) + 1) - erf(sqrt(2)*(-Delta/sigma + tau*(mu - sigma**2/2)/sigma)/(2*sqrt(tau))))*exp(-2*Delta*(mu - sigma**2/2)/sigma**2)/(4*(mu - sigma**2/2)**2) + sigma**2*((2*Delta*(mu - sigma**2/2)/sigma**2 + 1)*erf(sqrt(2)*(-Delta/sigma + tau*(mu - sigma**2/2)/sigma)/(2*sqrt(tau))) + exp(2*Delta*(mu - sigma**2/2)/sigma**2)*erf(sqrt(2)*(Delta/sigma + tau*(mu - sigma**2/2)/sigma)/(2*sqrt(tau))) - exp(2*Delta*(mu - sigma**2/2)/sigma**2) + 1)/(4*(mu - sigma**2/2)**2) + sqrt(2)*sigma*sqrt(tau)*exp(-(Delta/sigma + tau*(mu - sigma**2/2)/sigma)**2/(2*tau))/(2*sqrt(pi)*(mu - sigma**2/2)) - sqrt(2)*sigma*sqrt(tau)*exp(-(Delta/sigma - tau*(mu - sigma**2/2)/sigma)**2/(2*tau))/(2*sqrt(pi)*(mu - sigma**2/2)) + tau*erf(sqrt(2)*(Delta/sigma - tau*(mu - sigma**2/2)/sigma)/(2*sqrt(tau)))/2 + tau*erf(sqrt(2)

In [32]:
ev_fee_b = (theta / (1-E**(-delta/2) + l)) * (E**(mu * tau)) * (ev_fee_integral_fn(tau, delta/sig, mu_p/sig + sig) - limit(ev_fee_integral_fn(t, delta/sig, mu_p/sig + sig), t, 0))
ev_fee_b

theta*(sqrt(2)*sqrt(tau)*exp(-(Delta/sigma + tau*(sigma + (mu - sigma**2/2)/sigma))**2/(2*tau))/(2*sqrt(pi)*(sigma + (mu - sigma**2/2)/sigma)) - sqrt(2)*sqrt(tau)*exp(-(Delta/sigma - tau*(sigma + (mu - sigma**2/2)/sigma))**2/(2*tau))/(2*sqrt(pi)*(sigma + (mu - sigma**2/2)/sigma)) + tau*erf(sqrt(2)*(Delta/sigma - tau*(sigma + (mu - sigma**2/2)/sigma))/(2*sqrt(tau)))/2 + tau*erf(sqrt(2)*(Delta/sigma + tau*(sigma + (mu - sigma**2/2)/sigma))/(2*sqrt(tau)))/2 - (-Delta*exp(2*Delta*mu/sigma**2 + Delta) + Delta)/(2*mu*exp(2*Delta*mu/sigma**2 + Delta) + sigma**2*exp(2*Delta*mu/sigma**2 + Delta)) + ((2*Delta*(sigma + (mu - sigma**2/2)/sigma)/sigma - 1)*(exp(2*Delta*(sigma + (mu - sigma**2/2)/sigma)/sigma)*erf(sqrt(2)*(Delta/sigma + tau*(sigma + (mu - sigma**2/2)/sigma))/(2*sqrt(tau))) - exp(2*Delta*(sigma + (mu - sigma**2/2)/sigma)/sigma) + 1) - erf(sqrt(2)*(-Delta/sigma + tau*(sigma + (mu - sigma**2/2)/sigma))/(2*sqrt(tau))))*exp(-2*Delta*(sigma + (mu - sigma**2/2)/sigma)/sigma)/(4*(sigma + (m

In [33]:
ev_fee = ev_fee_a + ev_fee_b
ev_fee

theta*(sigma**2*((2*Delta*(mu - sigma**2/2)/sigma**2 - 1)*(exp(2*Delta*(mu - sigma**2/2)/sigma**2)*erf(sqrt(2)*(Delta/sigma + tau*(mu - sigma**2/2)/sigma)/(2*sqrt(tau))) - exp(2*Delta*(mu - sigma**2/2)/sigma**2) + 1) - erf(sqrt(2)*(-Delta/sigma + tau*(mu - sigma**2/2)/sigma)/(2*sqrt(tau))))*exp(-2*Delta*(mu - sigma**2/2)/sigma**2)/(4*(mu - sigma**2/2)**2) + sigma**2*((2*Delta*(mu - sigma**2/2)/sigma**2 + 1)*erf(sqrt(2)*(-Delta/sigma + tau*(mu - sigma**2/2)/sigma)/(2*sqrt(tau))) + exp(2*Delta*(mu - sigma**2/2)/sigma**2)*erf(sqrt(2)*(Delta/sigma + tau*(mu - sigma**2/2)/sigma)/(2*sqrt(tau))) - exp(2*Delta*(mu - sigma**2/2)/sigma**2) + 1)/(4*(mu - sigma**2/2)**2) + sqrt(2)*sigma*sqrt(tau)*exp(-(Delta/sigma + tau*(mu - sigma**2/2)/sigma)**2/(2*tau))/(2*sqrt(pi)*(mu - sigma**2/2)) - sqrt(2)*sigma*sqrt(tau)*exp(-(Delta/sigma - tau*(mu - sigma**2/2)/sigma)**2/(2*tau))/(2*sqrt(pi)*(mu - sigma**2/2)) + tau*erf(sqrt(2)*(Delta/sigma - tau*(mu - sigma**2/2)/sigma)/(2*sqrt(tau)))/2 + tau*erf(sqrt(2)

In [34]:
# checks out
simplify(diff(ev_fee_a, tau))

theta*(erf(sqrt(2)*(2*Delta - 2*mu*tau + sigma**2*tau)/(4*sigma*sqrt(tau))) + erf(sqrt(2)*(2*Delta + 2*mu*tau - sigma**2*tau)/(4*sigma*sqrt(tau))))*exp(3*Delta/2)/(2*(l*exp(3*Delta/2) + exp(3*Delta/2) - exp(Delta)))

In [35]:
# checks out too
simplify(diff(ev_fee_b * E**(-mu * tau), tau))

theta*(-erf(sqrt(2)*(-2*Delta + 2*mu*tau + sigma**2*tau)/(4*sigma*sqrt(tau))) + erf(sqrt(2)*(2*Delta + 2*mu*tau + sigma**2*tau)/(4*sigma*sqrt(tau))))*exp(Delta)/(2*(l*exp(Delta) - exp(Delta/2) + exp(Delta)))

In [36]:
# terms for principal before rebalancing
ev_principal_before_a = (E**(delta/2) + 1) * (E**(mu*tau) * Phi(d_minus - sig * sqrt(tau)) + 1 - Phi(d_plus))
ev_principal_before_a

(exp(Delta/2) + 1)*((erf(sqrt(2)*(-sigma*sqrt(tau) + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + 1/2)*exp(mu*tau) - erf(sqrt(2)*(Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2 + 1/2)

In [37]:
ev_principal_before_b = (1/(1-E**(-delta/2))) * (2 * E**((mu-sig**2/4) * (tau/2))) * (Phi(d_plus - sig * sqrt(tau) / 2) - Phi(d_minus - sig * sqrt(tau) / 2))
ev_principal_before_b

2*(-erf(sqrt(2)*(-sigma*sqrt(tau)/2 + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(-sigma*sqrt(tau)/2 + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(tau*(mu - sigma**2/4)/2)/(1 - exp(-Delta/2))

In [38]:
ev_principal_before_c = -(E**(-delta/2)/(1-E**(-delta/2))) * (Phi(d_plus) - Phi(d_minus) + E**(mu*tau) * (Phi(d_plus - sig * sqrt(tau)) - Phi(d_minus - sig * sqrt(tau))))
ev_principal_before_c

-((-erf(sqrt(2)*(-sigma*sqrt(tau) + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(-sigma*sqrt(tau) + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(mu*tau) - erf(sqrt(2)*(-Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2 + erf(sqrt(2)*(Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2)*exp(-Delta/2)/(1 - exp(-Delta/2))

In [39]:
ev_principal_before = ev_principal_before_a + ev_principal_before_b + ev_principal_before_c

In [40]:
# terms for swap fees paid on rebalance
ev_swap_fees_a = - (f/2) * (E**(delta/2) + 1) * (E**(mu * tau) * Phi(d_minus - sig * sqrt(tau)) + 1 - Phi(d_plus))
ev_swap_fees_a

-f*(exp(Delta/2) + 1)*((erf(sqrt(2)*(-sigma*sqrt(tau) + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + 1/2)*exp(mu*tau) - erf(sqrt(2)*(Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2 + 1/2)/2

In [41]:
ev_swap_fees_b = - (f/2) * (1/(E**(delta/2) - 1)) * (E**(mu * tau) * (Phi(d_plus - sig*sqrt(tau)) + Phi(d_minus - sig * sqrt(tau)) - 2 * Phi(-mu_p * tau / (sig * sqrt(tau)) - sig * sqrt(tau))) + 2*Phi(-mu_p * tau/(sig * sqrt(tau))) - Phi(d_plus) - Phi(d_minus))
ev_swap_fees_b

-f*((erf(sqrt(2)*(-sigma*sqrt(tau) + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(-sigma*sqrt(tau) + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 - erf(sqrt(2)*(-sigma*sqrt(tau) + sqrt(tau)*(-mu + sigma**2/2)/sigma)/2))*exp(mu*tau) - erf(sqrt(2)*(-Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2 - erf(sqrt(2)*(Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2 + erf(sqrt(2)*sqrt(tau)*(-mu + sigma**2/2)/(2*sigma)))/(2*(exp(Delta/2) - 1))

In [42]:
ev_swap_fees = ev_swap_fees_a + ev_swap_fees_b

In [43]:
# terms for slippage paid on rebalance
ev_slippage_a = -(l/4) * E**((mu + 3 * sig**2/4) * (tau / 2)) * (E**(delta/2) + 1)**2 * (E**(-mu*tau) * (1 - Phi(d_plus + sig * sqrt(tau)/2)) + E**(mu*tau) * Phi(d_minus - 3*sig*sqrt(tau) / 2))
ev_slippage_a

-l*((1/2 - erf(sqrt(2)*(sigma*sqrt(tau)/2 + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(-mu*tau) + (erf(sqrt(2)*(-3*sigma*sqrt(tau)/2 + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + 1/2)*exp(mu*tau))*(exp(Delta/2) + 1)**2*exp(tau*(mu + 3*sigma**2/4)/2)/4

In [44]:
ev_slippage_b = -(l/4) * E**((mu + 3 * sig**2/4) * (tau / 2)) * (1/(E**(delta/2) - 1)**2) * (E**(mu * tau) * (Phi(d_plus - 3 * sig * sqrt(tau) / 2) - Phi(d_minus - 3 * sig * sqrt(tau) / 2)) + E**(-mu*tau) * (Phi(d_plus + sig * sqrt(tau) / 2) - Phi(d_minus + sig * sqrt(tau) / 2)))
ev_slippage_b

-l*((-erf(sqrt(2)*(-3*sigma*sqrt(tau)/2 + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(-3*sigma*sqrt(tau)/2 + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(mu*tau) + (-erf(sqrt(2)*(sigma*sqrt(tau)/2 + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(sigma*sqrt(tau)/2 + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(-mu*tau))*exp(tau*(mu + 3*sigma**2/4)/2)/(4*(exp(Delta/2) - 1)**2)

In [45]:
ev_slippage_c = (l/2) * E**((mu + 3 * sig**2/4) * (tau / 2)) * (1/(E**(delta/2) - 1)**2) * (E**(-sig**2 * tau / 2)) * (Phi(d_plus - sig * sqrt(tau)/2) - Phi(d_minus - sig * sqrt(tau)/2))
ev_slippage_c

l*(-erf(sqrt(2)*(-sigma*sqrt(tau)/2 + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(-sigma*sqrt(tau)/2 + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(-sigma**2*tau/2)*exp(tau*(mu + 3*sigma**2/4)/2)/(2*(exp(Delta/2) - 1)**2)

In [46]:
ev_slippage = ev_slippage_a + ev_slippage_b + ev_slippage_c

In [47]:
ev_principal = ev_principal_before + ev_swap_fees + ev_slippage

In [48]:
rho_bar = ev_principal

In [49]:
psi_bar = ev_fee

In [50]:
# v_bar = ev of LP value
v_bar = ev_principal + ev_fee

In [51]:
v_bar_fn = Function('V')(delta)
v_bar_fn

V(Delta)

In [52]:
rho_bar_fn = Function('rho')(delta)
rho_bar_fn

rho(Delta)

In [53]:
psi_bar_fn = Function('psi')(delta)
psi_bar_fn

psi(Delta)

### Expansion about $\Delta = 0$

For second order expansion about $\Delta \to 0$, need the following terms at $0$:

- $V$
- $\partial_{\Delta} V$
- $\partial^2_{\Delta} V$

In [54]:
v_bar

-f*(exp(Delta/2) + 1)*((erf(sqrt(2)*(-sigma*sqrt(tau) + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + 1/2)*exp(mu*tau) - erf(sqrt(2)*(Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2 + 1/2)/2 - f*((erf(sqrt(2)*(-sigma*sqrt(tau) + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(-sigma*sqrt(tau) + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 - erf(sqrt(2)*(-sigma*sqrt(tau) + sqrt(tau)*(-mu + sigma**2/2)/sigma)/2))*exp(mu*tau) - erf(sqrt(2)*(-Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2 - erf(sqrt(2)*(Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2 + erf(sqrt(2)*sqrt(tau)*(-mu + sigma**2/2)/(2*sigma)))/(2*(exp(Delta/2) - 1)) - l*((1/2 - erf(sqrt(2)*(sigma*sqrt(tau)/2 + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(-mu*tau) + (erf(sqrt(2)*(-3*sigma*sqrt(tau)/2 + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + 1/2)*exp(mu*tau))*(exp(Delta/2) + 1)**2*exp(tau*(mu + 3*sigma**2/4)/2)/4 - l*((-erf(sqrt(

In [55]:
v_bar_fn

V(Delta)

In [56]:
limit(ev_fee, delta, 0)

0

In [57]:
simplify(limit(ev_principal_before, delta, 0))

-exp(mu*tau)*erf(sqrt(2)*sqrt(tau)*(2*mu + sigma**2)/(4*sigma)) + exp(mu*tau) + erf(sqrt(2)*sqrt(tau)*(2*mu - sigma**2)/(4*sigma)) + 1

In [58]:
simplify(limit(ev_swap_fees, delta, 0))

f*(exp(mu*tau)*erf(sqrt(2)*sqrt(tau)*(2*mu + sigma**2)/(4*sigma)) - exp(mu*tau) - erf(sqrt(2)*sqrt(tau)*(2*mu - sigma**2)/(4*sigma)) - 1)/2

In [59]:
simplify(limit(ev_slippage, delta, 0))

l*(exp(2*mu*tau)*erf(sqrt(2)*sqrt(tau)*(mu + sigma**2)/(2*sigma)) - exp(2*mu*tau) - erf(sqrt(2)*sqrt(tau)*(mu - sigma**2)/(2*sigma)) - 1)*exp(tau*(-4*mu + 3*sigma**2)/8)/2

In [60]:
diff(v_bar_fn, delta)

Derivative(V(Delta), Delta)